In [1]:
import cat4py as cat
import numpy as np
import os
import time

In [2]:
class Array(cat.Container):
    
    def __init__(self, dtype, pshape=None, filename=None, **kargs):
        self.dtype = np.dtype(dtype)
        kargs["itemsize"] = self.dtype.itemsize
        if pshape is not None:
            kargs["metalayers"] = {"numpy": {"dtype": str(self.dtype)}}
        self.kargs = kargs
        super(cat.Container, self).__init__(pshape, filename, **kargs)
    
    def __getitem__(self, key):    
        buff = cat.Container.__getitem__(self, key)
        ndim = self.ndim
        if ndim == 1:
            key = [key]

        key = list(key)

        for i, sl in enumerate(key):
            if type(sl) is not slice:
                key[i] = slice(sl, sl+1, None)

        start = [s.start if s.start is not None else 0 for s in key]
        stop = [s.stop if s.stop is not None else sh for s, sh in zip(key, self.shape)]
        shape = [sp - st for st, sp in zip(start, stop)]
        return np.frombuffer(buff, dtype=self.dtype).reshape(shape)
    
    
    def iter_write(self):
        return cat.Container.iter_write(self, self.dtype)
    
    def iter_read(self, blockshape):
        return cat.Container.iter_read(self, blockshape, self.dtype)
    
    def copy(self, pshape=None, filename=None, **kargs):
        arr = cat.Container.copy(self, pshape, filename, **kargs)
        arr.__class__ = Array
        arr.dtype = self.dtype
        return arr
    
    def __array__(self):
        return cat.Container.to_numpy(self, self.dtype)

def empty_array(dtype, shape, pshape=None, filename=None, **kargs):
    arr = Array(dtype, pshape, filename, **kargs)
    arr.updateshape(shape)
    return arr

def from_file(filename):
    arr = cat.from_file(filename)
    arr.__class__ = Array
    dtype = arr.get_metalayer("numpy")[b"dtype"]
    arr.dtype = np.dtype(dtype)
    return arr

def

In [3]:
a = empty_array(np.int32, shape=(32, 32), pshape=(16, 16), filename="notebook.cat4npy")

NameError: name 'kargs' is not defined

In [ ]:
for block, info in a.iter_write():
    block[:] = np.ones((info.shape), dtype=np.int32)

In [ ]:
b = a.copy(pshape=(12, 3), filename="notebook2.cat4npy")

In [ ]:
np.matmul(a, b)

In [ ]:
os.remove("notebook.cat4npy")
os.remove("notebook2.cat4npy")

In [ ]:
cat_array = empty_array(np.float64, shape=(2048, 2048), pshape=(2, 2048), filename="slice.cat4npy")

In [ ]:
for block, info in cat_array.iter_write():
    block[:] = np.random.randn(info.size).reshape(info.shape)

In [ ]:
numpy_array = cat_array.__array__()
np.save("slice", numpy_array)

In [ ]:
t1 = time.time()
new_numpy_array = np.load("slice.npy")
numpy_slice = new_numpy_array[2:3, :]
t2 = time.time()
t_numpy = t2 - t1
print(t_numpy)

In [ ]:
t1 = time.time()
new_cat_array = from_file("slice.cat4npy")
cat_slice = new_cat_array[2:3, :]
t2 = time.time()
t_cat = t2 - t1
print(t_cat)

In [ ]:
print(t_numpy / t_cat)

In [ ]:
np.testing.assert_almost_equal(cat_slice, numpy_slice)

In [ ]:
os.remove("slice.cat4npy")
os.remove("slice.npy")